# NLP20●●からの論文情報抽出
## 抽出対象
- タイトル
- 著者
- 論文のURL
- PDF中のintroduction
- カテゴリ
    - タスク
    - ポスター

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pdfminer.pdfpage import PDFPage
import re

## 言語処理学会の発表論文集のページの分析
- URL
    - https://www.anlp.jp/proceedings/annual_meeting/20●●/ のようになっている
    - ex.) 2019なら https://www.anlp.jp/proceedings/annual_meeting/2019/

- ページのデザイン
    - 2014 ~ 2019が現行のデザイン
    - ~2013は旧デザイン
    - oral発表のセッション名はclass="program_oral"で設定されているみたい.
        - ただし，年度によって，セッション名が違う．
        - タスクが多様化したり，増えたりすることが理由か．これの対処は必要そう
        - セッション名をそのまんま分類先のクラスとして考える必要はなく，ある程度まとめてもいいと思う．
            - 例えば，知識獲得，情報抽出，固有表現抽出はIE（Information Extraction）クラスにするみたいな

- 2014〜は以下の方法で取得できるが，〜2013は取得できない．抽出したいなら，エラーの検証が必要

In [2]:
r = requests.get('https://www.anlp.jp/proceedings/annual_meeting/2019/')
# r = requests.get('https://www.anlp.jp/proceedings/annual_meeting/2013/')

In [3]:
r.encoding = r.apparent_encoding
r

<Response [200]>

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')

### 発表情報の抽出方法の検討
- class=session1とclass=session2で発表は抽出できそう
    - session1と2の違いは，背景色
    - session1：白，session2：グレー
- フィルタリング対象
    - チュートリアル
        - チュートリアルはidがTから始まるため，これを見ればフィルタリング可能
    - 招待講演
        - 招待講演はidがinvited+数字 か I+数字で決まるため(2019のみ)，これを利用する
    - テーマセッション
        - テーマセッションは文字列中に”テーマセッション”と書かれるため，これを利用する
    - ワークショップ
        - idにworkshopという文字列が含まれている
    - その他
        - sympo（シンポジウムかな)
        - sponser_evening
- まとめて，class=session_titleの中で，id=英大文字 + 数字出ないものをフィルタリングし，タイトルの文字列が英大文字で始まらないもの（チュートリアルと招待講演）を除けば，論文集は抽出できそう

In [5]:
session1 = soup.find_all(class_='session1')
session2 = soup.find_all(class_='session2')

In [6]:
print(session1[0].find(class_='session_title'))
print(session1[-1].find(class_='session_title'))

<span class="session_title" id="T1">チュートリアル(1)</span>
<span class="session_title" id="E6">E6:知識獲得・情報抽出(3)</span>


In [7]:
for s1 in session1:
    session_name = s1.find(class_='session_title')
    if re.search(r'[A-Z]\d+', session_name['id']) is None:
        continue
    if re.search(r'[A-Z]\d+', session_name.text) is None:
        continue
    print(session_name)

<span class="session_title" id="A1">A1:機械翻訳(1)</span>
<span class="session_title" id="C1">C1:実験データに基づく言語学(1)</span>
<span class="session_title" id="E1">E1:知識獲得・情報抽出(1)</span>
<span class="session_title" id="A2">A2:機械翻訳(2)</span>
<span class="session_title" id="C2">C2:言語資源(1)</span>
<span class="session_title" id="E2">E2:音声言語処理</span>
<span class="session_title" id="A3">A3:埋め込み表現(1)</span>
<span class="session_title" id="C3">C3:実験データに基づく言語学(2)</span>
<span class="session_title" id="E3">E3:知識獲得・情報抽出(2)</span>
<span class="session_title" id="P1">P1:ポスター(1)</span>
<span class="session_title" id="P3">P3:ポスター(2)</span>
<span class="session_title" id="A4">A4:テーマセッション: 実世界にグラウンドされた言語処理</span>
<span class="session_title" id="C4">C4:テーマセッション: 言語教育と言語処理の接点</span>
<span class="session_title" id="E4">E4:テーマセッション: 試験問題をベンチマークとする言語処理</span>
<span class="session_title" id="A5">A5:機械翻訳(3)</span>
<span class="session_title" id="C5">C5:言語資源(2)</span>
<span class="session_title" id="E5">E5:機械学習</span>
<sp

#### ただし，セッションを抽出しても，セッションに紐づく論文はうまく抽出できない

In [8]:
s1

<div class="session1">
<div class="session_header"><span class="session_title" id="E6">E6:知識獲得・情報抽出(3)</span>
　　3月15日(金) 15:00-16:40   ES024　　座長: 山田一郎(NHK)<br/></div>
<table>
<tr><td class="pid"><span id="E6-1">E6-1</span></td>
<td><span class="title">Conditional VAEに基づく多様性を考慮したイベント予測</span></td></tr>
<tr><td><a href="pdf_dir/E6-1.pdf"><img src="html/img/pdf.png"/></a></td>
<td>○清丸寛一, 大村和正, 村脇有吾, 河原大輔, 黒橋禎夫 (京大)</td></tr>
<tr><td class="pid"><span id="E6-2">E6-2</span></td>
<td><span class="title">ニューラルネットワークを用いたトピック遷移モデリングに関する検討</span></td></tr>
<tr><td><a href="pdf_dir/E6-2.pdf"><img src="html/img/pdf.png"/></a></td>
<td>○内田脩斗, 吉川大弘, 古橋武 (名大)</td></tr>
<tr><td class="pid"><span id="E6-3">E6-3</span></td>
<td><span class="title">単語の分散表現を用いた日本語イベント連鎖の自動構築</span></td></tr>
<tr><td><a href="pdf_dir/E6-3.pdf"><img src="html/img/pdf.png"/></a></td>
<td>○瀧下祥, Rafal Rzepka, 荒木健治 (北大)</td></tr>
<tr><td class="pid"><span id="E6-4">E6-4</span></td>
<td><span class="title">TextRankと依存関係情報の組み合わせに

### セッションに基づく論文の抽出

In [ ]:
for tr in soup.find_all('tr'):
    if tr.find_all('td') != []:
        # セッション番号: session 論文タイトル: title
        pid = tr.find('td', class_='pid')
        if pid is not None and pid.find('span') is not None:
            paper_info = tr.find_all('td')
            session = paper_info[0].text[:2]
            title = paper_info[1].text
            print(session, title)
        # 論文URL: url
        elif tr.find('a') is not None and tr.find('a')['href'][0] != '#':
            print(tr.find('a'))
            url = tr.find('a')['href']
            authors_str = re.sub(r'\s\(.+\)', '', tr.find_all('td')[1].text)
            authors = authors_str.replace('○', '').split(', ')
            print(url)
            print(authors)

### PDFからのintroductionの抽出
- pdfからの情報抽出には，pdfminerを用いる
- pdfからの情報抽出は以下のように，くっついて出力されるので，ルールにより，ある程度のところで切り取る
    - ルール
        - 1 + ['はじめに', '序論', '背景', '背景と目的', 'Introduction']を探す
            - ある場合はそれを始点として情報抽出
            - ない場合は，1ページ目全てを保存する
        - ".2" or "。2"を探す
            - ある場合は，それを終点として情報抽出
            - ない場合は，1ページ目全てを保存する

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from io import BytesIO

r = requests.get(
    # 'https://www.anlp.jp/proceedings/annual_meeting/2019/pdf_dir/A1-1.pdf'
    #'https://www.anlp.jp/proceedings/annual_meeting/2019/pdf_dir/E3-3.pdf'
    #'https://www.anlp.jp/proceedings/annual_meeting/2019/pdf_dir/P1-1.pdf'
    'https://www.anlp.jp/proceedings/annual_meeting/2019/pdf_dir/P1-36.pdf'
)
# BytesIOとStringIOはbyteやstringをIOオブジェクト，つまりファイルオープンしたオブジェクトと等価にする
instr = BytesIO()
instr.write(r.content)
outstr = StringIO()
manager = PDFResourceManager()
laparams = LAParams()
laparams.detect_vertical = True
with TextConverter(manager, outstr, codec='utf-8', laparams=laparams) as device:
    interpreter = PDFPageInterpreter(manager, device)
    for page in PDFPage.get_pages(instr, set(), maxpages=1, caching=True, check_extractable=True):
        interpreter.process_page(page)
        onepage = outstr.getvalue()
        intro = ''
        for chap in ['はじめに', '序論', '背景', '背景と目的', 'Introduction']:
            chap_name = '1{}'.format(chap)
            if chap_name in onepage:
                if '．2' in onepage:
                    intro = onepage[onepage.find(chap_name) + len(chap_name): onepage.find('．2')]
                elif '.2' in onepage:
                    intro = onepage[onepage.find(chap_name) + len(chap_name): onepage.find('.2')]
                elif '。2' in onepage:
                    intro = onepage[onepage.find(chap_name) + len(chap_name): onepage.find('。2')]
                break
        intro = onepage if intro == '' else intro
        print(onepage)
        print('-----------------------------')
        print(intro)